#Importing drive so as to access train and test files from Google Drive

*  Google Colab provides a GPU which helps in training the model faster




In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# **Importing required libraries**
****

In [2]:
import pandas as pd
import numpy as np


import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.regularizers import L1L2


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


import string
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer, SnowballStemmer


Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


##**Reading Train and Test Files from google drive and transforming them into a pandas dataframe**

In [0]:
train_data = pd.read_csv('gdrive/My Drive/train_file.csv')#, encoding='ISO-8859-1')
test_data = pd.read_csv('gdrive/My Drive/test_file.csv')#, encoding='utf8')

**Changing the names of the columns into a lower case letters and white spaces are being replaced with underscore(''_")**

In [0]:
train_data.columns = train_data.columns.str.lower().str.rstrip()
train_data.columns = train_data.columns.str.replace(' ', '_')

##Data Analysis


*   Checking the length of each string present in the columns, Title and HeadLine, of the train_data dataframe
*   Checking the existence of Null values in the columns of the train dataset
*   Checking the Count of the NULL Values in each column of the train dataset
*   Imputed the NULL Values of categorical column with Mode (Most Frequent)
*   Shape of the train dataset and test dataset
*   No : of Words, Unique_words, Stop_words in the columns, Title and Headline of the each dataframe
*   Mean Length of words in the columns, Title and Headline, in each dataframe
*   Unique values over the index axis in train_data dataframe

 

In [5]:
max(train_data["headline"].str.len()), max(train_data["title"].str.len())

(446, 170)

In [6]:
train_data.head()

,idlink,title,headline,source,topic,publishdate,facebook,googleplus,linkedin,sentimenttitle,sentimentheadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084


In [7]:
test_data.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn
0,tFrqIR6Chj,Sliding Economy: FG fights back with N3trn TSA...,With the 2016 budget now passed by the Nationa...,BusinessDay,economy,2016-03-29 01:41:12,0,0,1
1,DVAaGErjlF,Microsoft shows how HoloLens can bring distant...,A recent Microsoft Research video shows how th...,Daily Mail,microsoft,2016-03-29 01:41:27,121,2,13
2,OT9UIZm5M2,"Microsoft’s Twitter Robot Praises Hitler, Trum...",* Microsoft teamed with Bing to create TayTwee...,EURweb,microsoft,2016-03-29 01:47:00,12,1,0
3,lflGp3q2Fj,Flood of Central Bank Moves Can't Get World Ec...,Central bankers have managed to steer the worl...,Bloomberg via Yahoo! Finance,economy,2016-03-29 02:00:00,0,0,3
4,zDYG0SoovZ,USD/JPY: bears lining up on mixed U.S. economy...,"However, this streak of seven-day gains might ...",FXStreet,economy,2016-03-29 02:01:07,3,0,0


In [0]:
train_data.drop(['idlink','publishdate'],axis=1,inplace=True)

#test_data.drop(['IDLink', 'PublishDate'],axis=1,inplace=True)

In [9]:
train_data.dtypes

title                 object
headline              object
source                object
topic                 object
facebook               int64
googleplus             int64
linkedin               int64
sentimenttitle       float64
sentimentheadline    float64
dtype: object

In [10]:
train_data.isnull().any()

title                False
headline             False
source                True
topic                False
facebook             False
googleplus           False
linkedin             False
sentimenttitle       False
sentimentheadline    False
dtype: bool

In [0]:
train_data['source'].value_counts(), train_data['topic'].value_counts()

In [0]:
# Filling the NULL Values in Source with Mode

train_data['source'].fillna(value=train_data['source'].value_counts().index[0],inplace =True)

In [11]:
train_data.isnull().sum()

title                  0
headline               0
source               175
topic                  0
facebook               0
googleplus             0
linkedin               0
sentimenttitle         0
sentimentheadline      0
dtype: int64

In [12]:
train_data.shape, test_data.shape

((55932, 9), (37288, 7))

In [0]:
train_data['headline_length'] = train_data["headline"].str.len()
train_data['title_length'] = train_data["title"].str.len()

In [0]:
# No. of words in the columns, Title and Headline, in each dataframe

train_data["num_words_title"] = train_data["title"].apply(lambda x: len(str(x).split()))
test_data["num_words_title"] = test_data["Title"].apply(lambda x: len(str(x).split()))

train_data["num_words_hLine"] = train_data["headline"].apply(lambda x: len(str(x).split()))
test_data["num_words_hLine"] = test_data["Headline"].apply(lambda x: len(str(x).split()))

In [0]:
# No. of Unique Words in the columns, Title and Headline, in each dataframe

train_data["num_unique_words_title"] = train_data["title"].apply(lambda x: len(str(x).split()))
test_data["num_unique_words_title"] = test_data["Title"].apply(lambda x: len(str(x).split()))

train_data["num_unique_words_hLine"] = train_data["headline"].apply(lambda x: len(str(x).split()))
test_data["num_unique_words_hLine"] = test_data["Headline"].apply(lambda x: len(str(x).split()))

In [15]:
# No. of Stop_Words in the columns, Title and Headline, in each dataframe

nltk.download('stopwords')
from nltk.corpus import stopwords

eng_stopwords = set(stopwords.words("english"))

train_data["num_stop_words_title"] = train_data["title"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
train_data["num_stop_words_hLine"] = train_data['headline'].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))


test_data["num_stop_words_title"] = test_data["Title"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_data["num_stop_words_title"] = test_data["Headline"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
# Mean Length of words in the columns, Title and Headline, in each dataframe

train_data["mean_word_len_title"] = train_data['title'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
train_data["mean_word_len_hLine"] = train_data['headline'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

test_data["mean_word_len_title"] = test_data['Title'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data["mean_word_len_hLine"] = test_data['Headline'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [17]:
train_data.head()

,title,headline,source,topic,facebook,googleplus,linkedin,sentimenttitle,sentimentheadline,num_words_title,num_words_hLine,num_unique_words_title,num_unique_words_hLine,num_stop_words_title,num_stop_words_hLine,mean_word_len_title,mean_word_len_hLine
0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,-1,-1,-1,0.000000,-0.053300,7,22,7,22,1,8,6.000000,4.818182
1,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,-1,-1,-1,0.208333,-0.156386,9,22,9,22,5,7,3.888889,5.272727
2,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,-1,-1,-1,-0.425210,0.139754,8,20,8,20,2,7,5.125000,5.900000
3,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,-1,-1,-1,0.000000,0.026064,5,23,5,23,1,6,4.200000,6.652174
4,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,-1,-1,-1,0.000000,0.141084,8,27,8,27,1,11,5.625000,5.333333


In [18]:
# Unique values over the index axis in train_data dataframe

for i in train_data.columns:
    print(i,' : ',train_data[i].nunique())

title  :  48963
headline  :  52112
source  :  4753
topic  :  4
facebook  :  2166
googleplus  :  273
linkedin  :  648
sentimenttitle  :  10014
sentimentheadline  :  27265
num_words_title  :  24
num_words_hLine  :  79
num_unique_words_title  :  24
num_unique_words_hLine  :  79
num_stop_words_title  :  12
num_stop_words_hLine  :  43
mean_word_len_title  :  497
mean_word_len_hLine  :  2353


In [0]:
train_data['source'].value_counts(), train_data['topic'].value_counts()

In [0]:
test_data['Source'].value_counts(), test_data['Topic'].value_counts()

## Read the required columns for Predictions




*   Consider only 'Title' column for predictions of SentimentTitle

*   Consider only 'HeadLine' column for predictions of SentimentHeadLine



In [0]:
train_data_title = train_data[train_data['title'].notnull()]
train_data_hLine = train_data[train_data['headline'].notnull()]

In [0]:
X_train_title = train_data_title['title']
Y_train_title = train_data_title['sentimenttitle']
X_train_hLine = train_data_hLine['headline']
Y_train_hLine = train_data_hLine['sentimentheadline']

# Preprocess text



*   Remove punctuations from data.
*   Apply lemmatization on words. Convert each word to its lemma.




---





In [0]:
#stemmer = SnowballStemmer("english")
lemma = WordNetLemmatizer()

def preprocess_data(X_train, lemma):
    preprocessed_data = []
    trans = str.maketrans('—/(){}-', ' ' * 7)
    trans_punc = str.maketrans('', '', string.punctuation)

    for text in X_train:
        #print(text)
        text = text.lower().translate(trans)
        text = text.translate(trans_punc)
        text = re.sub(r"\'s", " is ", text)
        text = re.sub(r"n't", " not ", text)
        text = [lemma.lemmatize(word) for word in text.split()]
        text = [x for x in text if not any(c.isdigit() for c in x)]
        preprocessed_data.append(' '.join(text))
        #print(text)

    return preprocessed_data

In [0]:
preprocessed_data_t = preprocess_data(X_train_title, lemma)
preprocessed_data_h = preprocess_data(X_train_hLine, lemma)

# Data Preparation for Keras Model Building



*  **Initialize Tokenizer Objects**







In [0]:
t_t = Tokenizer()

t_t.fit_on_texts(preprocessed_data_t)
encoded_lines_t = t_t.texts_to_sequences(preprocessed_data_t)

In [0]:
t_h = Tokenizer()

t_h.fit_on_texts(preprocessed_data_h)
encoded_lines_h = t_h.texts_to_sequences(preprocessed_data_h)

In [0]:
number_of_words_title = len(t_t.word_index) + 1
number_of_words_hLine = len(t_h.word_index) + 1

title_max_length_of_input = 200
hLine_max_length_of_input = 500

embedding_vector_length = 128


*   **Sequence Padding**

*   **Train_Validation Split : Splitting the training data into train data and validation data**

  *   Validation set plays a major role in improving the performance of the model.


In [0]:
X_train_t = sequence.pad_sequences(encoded_lines_t, title_max_length_of_input)
X_train_t, X_valid_t, y_train_t, y_valid_t = train_test_split(X_train_t, Y_train_title, test_size=0.2, random_state=13)


X_train_h = sequence.pad_sequences(encoded_lines_h, hLine_max_length_of_input)
X_train_h, X_valid_h, y_train_h, y_valid_h = train_test_split(X_train_h, Y_train_hLine, test_size=0.2, random_state=13)



# Model Building for Predictions



*   **TModel for predicting the SentimentTitle**
  *    Compile the TModel
  *    Summary of the TModel : Showing the architecture and Total Parameters of the TModel




In [0]:
adam = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [28]:
Tmodel = Sequential()
Tmodel.add(Embedding(number_of_words_title, embedding_vector_length, input_length = title_max_length_of_input))
Tmodel.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
Tmodel.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
Tmodel.add(LSTM(50))
Tmodel.add(Dense(1, activation = 'linear'))
Tmodel.compile(loss='mean_absolute_error', optimizer= adam, metrics=['mean_absolute_error'])
Tmodel.summary()

W0708 16:21:06.727590 139724375963520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 16:21:06.730935 139724375963520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 16:21:06.733603 139724375963520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0708 16:21:06.863080 139724375963520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0708 16:21:06.873550 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          2842368   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 100)          91600     
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 100)          80400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 3,044,619
Trainable params: 3,044,619
Non-trainable params: 0
_________________________________________________________________


*   **HModel for predicting the SentimentHeadLine**
  *    Compile the HModel
  *    Summary of the HModel : Showing the architecture and Total Parameters of the HModel


In [29]:
Hmodel = Sequential()
Hmodel.add(Embedding(number_of_words_hLine, embedding_vector_length, input_length = hLine_max_length_of_input))
Hmodel.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
Hmodel.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
Hmodel.add(LSTM(50))
Hmodel.add(Dense(1, activation = 'linear'))
Hmodel.compile(loss='mean_absolute_error', optimizer= adam, metrics=['mean_absolute_error'])
Hmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 128)          4918400   
_________________________________________________________________
lstm_4 (LSTM)                (None, 500, 100)          91600     
_________________________________________________________________
lstm_5 (LSTM)                (None, 500, 100)          80400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 5,120,651
Trainable params: 5,120,651
Non-trainable params: 0
_________________________________________________________________


# Fitting the Train data to the Model



*   **Fitting the train data of title to the TModel for predicting the SentimentTitle** 



In [30]:
Tmodel.fit(X_train_t, y_train_t, batch_size = 1024, epochs = 40, validation_data=(X_valid_t, y_valid_t),verbose = True, shuffle = True)


W0708 16:21:09.178352 139724375963520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0708 16:21:11.146214 139724375963520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 44745 samples, validate on 11187 samples
Epoch 1/40
44745/44745 [==============================] - 55s 1ms/step - loss: 0.0818 - mean_absolute_error: 0.0818 - val_loss: 0.0632 - val_mean_absolute_error: 0.0632
Epoch 2/40
44745/44745 [==============================] - 49s 1ms/step - loss: 0.0548 - mean_absolute_error: 0.0548 - val_loss: 0.0544 - val_mean_absolute_error: 0.0544
Epoch 3/40
44745/44745 [==============================] - 48s 1ms/step - loss: 0.0465 - mean_absolute_error: 0.0465 - val_loss: 0.0525 - val_mean_absolute_error: 0.0525
Epoch 4/40
44745/44745 [==============================] - 49s 1ms/step - loss: 0.0421 - mean_absolute_error: 0.0421 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 5/40
44745/44745 [==============================] - 48s 1ms/step - loss: 0.0383 - mean_absolute_error: 0.0383 - val_loss: 0.0482 - val_mean_absolute_error: 0.0482
Epoch 6/40
44745/44745 [==============================] - 47s 1ms/step - loss: 0.0352 - mean_absolute_err

*   **Fitting the train data of title to the TModel for predicting the SentimentTitle**

In [31]:
Hmodel.fit(X_train_h, y_train_h, batch_size = 1024, epochs = 40, validation_data=(X_valid_h, y_valid_h),verbose = True)

Train on 44745 samples, validate on 11187 samples
Epoch 1/40
44745/44745 [==============================] - 138s 3ms/step - loss: 0.1380 - mean_absolute_error: 0.1380 - val_loss: 0.1101 - val_mean_absolute_error: 0.1101
Epoch 2/40
44745/44745 [==============================] - 133s 3ms/step - loss: 0.1078 - mean_absolute_error: 0.1078 - val_loss: 0.1039 - val_mean_absolute_error: 0.1039
Epoch 3/40
44745/44745 [==============================] - 134s 3ms/step - loss: 0.0866 - mean_absolute_error: 0.0866 - val_loss: 0.0759 - val_mean_absolute_error: 0.0759
Epoch 4/40
44745/44745 [==============================] - 134s 3ms/step - loss: 0.0636 - mean_absolute_error: 0.0636 - val_loss: 0.0611 - val_mean_absolute_error: 0.0611
Epoch 5/40
44745/44745 [==============================] - 134s 3ms/step - loss: 0.0511 - mean_absolute_error: 0.0511 - val_loss: 0.0558 - val_mean_absolute_error: 0.0558
Epoch 6/40
44745/44745 [==============================] - 134s 3ms/step - loss: 0.0447 - mean_absolu

# Conversion of Text Form Test Data into Vector Form

In [0]:
X_test_title = test_data['Title']
X_test_title = preprocess_data(X_test_title, lemma)
X_test_title = t_t.texts_to_sequences(X_test_title)
X_test_title = sequence.pad_sequences(X_test_title, title_max_length_of_input)


X_test_hLine = test_data['Headline']
X_test_hLine = preprocess_data(X_test_hLine, lemma)
X_test_hLine = t_h.texts_to_sequences(X_test_hLine)
X_test_hLine = sequence.pad_sequences(X_test_hLine, hLine_max_length_of_input)

# Predict the output of the Test Data

In [0]:
T_output = Tmodel.predict(X_test_title, 1024).flatten()

H_output = Hmodel.predict(X_test_hLine, 1024).flatten()

# Importing the results into the CSV File

In [0]:
test_data_df = pd.DataFrame({'IDLink': test_data['IDLink'],'SentimentTitle': T_output, 'SentimentHeadline':H_output})
test_data_df.to_csv('gdrive/My Drive/Approach_1_file.csv', index=False, columns=['IDLink','SentimentTitle', 'SentimentHeadline'])
test_data_df.to_csv('Output_file.csv', index=False, columns=['IDLink','SentimentTitle', 'SentimentHeadline'])